# 13F Info Table Analytics

This notebook analyzes 13F-HR holdings data across different quarters and institutions.

**Objectives:**
- Read all Excel files from `data/extracted_13F_HR/` folders
- Extract year-quarter from filenames (format: YYYYMMDD)
- Apply class_title transformation using `ClassTitleTransform`
- Analyze holdings by category and time period

## Setup and Imports

In [ ]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src'))

from data_transformation.class_title_transform import ClassTitleTransform

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Helper Functions

In [ ]:
def extract_year_quarter(filename):
    """
    Extract year and quarter from filename in format YYYYMMDD.xlsx
    
    Examples:
    - 20240930.xlsx -> (2024, 'Q3')
    - 20241231.xlsx -> (2024, 'Q4')
    """
    # Remove .xlsx extension
    date_str = filename.replace('.xlsx', '')
    
    # Parse date
    year = int(date_str[:4])
    month = int(date_str[4:6])
    
    # Determine quarter based on month
    if month in [1, 2, 3]:
        quarter = 'Q1'
    elif month in [4, 5, 6]:
        quarter = 'Q2'
    elif month in [7, 8, 9]:
        quarter = 'Q3'
    else:  # 10, 11, 12
        quarter = 'Q4'
    
    return year, quarter

def get_year_quarter_label(filename):
    """Get formatted year-quarter label (e.g., '2024-Q3')"""
    year, quarter = extract_year_quarter(filename)
    return f"{year}-{quarter}"

# Test the function
test_files = ['20240930.xlsx', '20241231.xlsx', '20250331.xlsx']
for f in test_files:
    print(f"{f} -> {get_year_quarter_label(f)}")

## Load All Data

In [ ]:
# Initialize paths
base_dir = Path.cwd().parent
data_dir = base_dir / 'data' / 'extracted_13F_HR'

# Initialize transformer
transformer = ClassTitleTransform()

# List to store all dataframes
all_data = []

print("Loading data from all institutions and quarters...\n")

# Iterate through all institution folders
for institution_folder in sorted(data_dir.iterdir()):
    if not institution_folder.is_dir():
        continue
    
    institution_name = institution_folder.name
    print(f"Processing: {institution_name}")
    
    # Get all Excel files
    xlsx_files = [f for f in institution_folder.glob('*.xlsx') 
                  if not f.name.startswith('~$')]
    
    for xlsx_file in sorted(xlsx_files):
        try:
            # Extract year-quarter
            year_quarter = get_year_quarter_label(xlsx_file.name)
            
            # Read Excel file
            df = pd.read_excel(xlsx_file)
            
            # Add metadata columns
            df['institution'] = institution_name
            df['year_quarter'] = year_quarter
            df['file_date'] = xlsx_file.name.replace('.xlsx', '')
            
            # Apply class_title transformation
            if 'class_title' in df.columns:
                df['class_category'] = transformer.transform(df['class_title'])
            
            all_data.append(df)
            print(f"  ✓ {xlsx_file.name} ({year_quarter}): {len(df):,} holdings")
            
        except Exception as e:
            print(f"  ✗ Error reading {xlsx_file.name}: {str(e)}")

# Combine all data
combined_df = pd.concat(all_data, ignore_index=True)

print(f"\n{'='*60}")
print(f"Total records loaded: {len(combined_df):,}")
print(f"Institutions: {combined_df['institution'].nunique()}")
print(f"Quarters: {combined_df['year_quarter'].nunique()}")
print(f"{'='*60}")

## Data Overview

In [ ]:
# Display first few rows
print("Sample data:")
combined_df.head()

In [ ]:
# Display column information
print("Dataset Info:")
combined_df.info()

In [ ]:
# Check unique institutions and quarters
print("Institutions:")
print(combined_df['institution'].value_counts())
print("\nYear-Quarters:")
print(combined_df['year_quarter'].value_counts().sort_index())

## Class Category Analysis

In [ ]:
# Overall category distribution
print("Top 20 Security Categories by Count:")
category_counts = combined_df['class_category'].value_counts()
print(category_counts.head(20))

# Visualize
plt.figure(figsize=(14, 8))
category_counts.head(20).plot(kind='barh')
plt.xlabel('Number of Holdings')
plt.ylabel('Security Category')
plt.title('Top 20 Security Categories Across All Holdings')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Category distribution by institution
print("Category Distribution by Institution:")
category_by_inst = pd.crosstab(combined_df['institution'], 
                                combined_df['class_category'])
print(category_by_inst.head())

# Show top categories for each institution
for institution in combined_df['institution'].unique():
    print(f"\nTop 10 categories for {institution.upper()}:")
    inst_data = combined_df[combined_df['institution'] == institution]
    print(inst_data['class_category'].value_counts().head(10))

## Time Series Analysis

In [ ]:
# Holdings count over time by institution
holdings_over_time = combined_df.groupby(['year_quarter', 'institution']).size().unstack(fill_value=0)

print("Holdings Count by Quarter and Institution:")
print(holdings_over_time)

# Plot
holdings_over_time.plot(kind='bar', figsize=(12, 6))
plt.xlabel('Year-Quarter')
plt.ylabel('Number of Holdings')
plt.title('Holdings Count Over Time by Institution')
plt.legend(title='Institution')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Category trends over time
# Get top 10 categories overall
top_categories = combined_df['class_category'].value_counts().head(10).index.tolist()

# Filter for top categories
top_cat_df = combined_df[combined_df['class_category'].isin(top_categories)]

# Count by quarter and category
category_trends = top_cat_df.groupby(['year_quarter', 'class_category']).size().unstack(fill_value=0)

print("Top 10 Categories Over Time:")
print(category_trends)

# Plot
category_trends.plot(kind='line', figsize=(14, 8), marker='o')
plt.xlabel('Year-Quarter')
plt.ylabel('Number of Holdings')
plt.title('Top 10 Security Categories Trend Over Time')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Value Analysis (if available)

In [ ]:
# Check if value column exists
value_col = None
for col in ['value', 'market_value', 'total_value', 'value_x1000']:
    if col in combined_df.columns:
        value_col = col
        break

if value_col:
    print(f"Analyzing values using column: {value_col}\n")
    
    # Top categories by total value
    category_value = combined_df.groupby('class_category')[value_col].sum().sort_values(ascending=False)
    
    print("Top 20 Categories by Total Value:")
    print(category_value.head(20))
    
    # Visualize
    plt.figure(figsize=(14, 8))
    category_value.head(20).plot(kind='barh')
    plt.xlabel(f'Total {value_col.replace("_", " ").title()}')
    plt.ylabel('Security Category')
    plt.title('Top 20 Security Categories by Total Value')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("No value column found in the dataset.")
    print(f"Available columns: {', '.join(combined_df.columns)}")

## Summary Statistics

In [ ]:
# Create summary table
summary = combined_df.groupby(['institution', 'year_quarter']).agg({
    'class_category': 'count',
    'class_title': 'nunique'
}).rename(columns={
    'class_category': 'total_holdings',
    'class_title': 'unique_securities'
})

print("Summary by Institution and Quarter:")
print(summary)

# Category diversity by institution-quarter
category_diversity = combined_df.groupby(['institution', 'year_quarter'])['class_category'].nunique()
summary['unique_categories'] = category_diversity

print("\nWith Category Diversity:")
print(summary)

## Export Processed Data

In [ ]:
# Save combined data with categories
output_dir = base_dir / 'notebook' / 'data'
output_dir.mkdir(exist_ok=True)

output_file = output_dir / 'combined_holdings_with_categories.parquet'
combined_df.to_parquet(output_file, index=False)
print(f"Saved processed data to: {output_file}")

# Also save summary
summary_file = output_dir / 'holdings_summary.csv'
summary.to_csv(summary_file)
print(f"Saved summary to: {summary_file}")